In [16]:
import numpy as np
import pandas as pd
import networkx as nx
import osmnx as ox
ox.config(use_cache=True, log_console=True)
import warnings
warnings.filterwarnings('ignore')

In [8]:
from here_location_services import LS
import os
from here_location_services.config.matrix_routing_config import(WorldRegion, MATRIX_ATTRIBUTES, BoundingBoxRegion, AutoCircleRegion,)

In [4]:
LS_API_KEY = os.environ.get("LS_API_KEY")

In [6]:
ls= LS(api_key=LS_API_KEY)

In [10]:
origins = [
{"lat": 49.747336, "lng": 9.805835},
{"lat": 49.748455, "lng": 9.848022}
]
region_definition = AutoCircleRegion()
matrix_attributes = [MATRIX_ATTRIBUTES.travelTimes]
result = ls.matrix(
origins=origins,
region_definition=region_definition,
matrix_attributes=matrix_attributes,
async_req = True
)
result.matrix

{'numOrigins': 2, 'numDestinations': 2, 'travelTimes': [0, 102, 1009, 0]}

In [14]:
result.matrix["travelTimes"]

[0, 102, 1009, 0]

In [17]:
mylist = result.matrix["travelTimes"]
np_array = np.array(mylist)
reshaped_array = np.reshape(np_array, (2,2))
mydf = pd.DataFrame(reshaped_array)
mydf

,0,1
0,0,102
1,1009,0


In [2]:
place = {"city": "Wuerzburg", "country": "Germany"}

In [3]:
G = ox.graph_from_place(place, network_type = "drive")
Gs = ox.utils_graph.get_largest_component(G, strongly = True)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(Gs)
gdf_nodes

,y,x,street_count,highway,geometry
osmid,,,,,
10799058,49.751452,9.926904,3,NaN,POINT (9.92690 49.75145)
10799066,49.753819,9.928881,3,NaN,POINT (9.92888 49.75382)
10799083,49.751798,9.927970,3,NaN,POINT (9.92797 49.75180)
10799085,49.751011,9.927733,3,NaN,POINT (9.92773 49.75101)
10799087,49.754406,9.928837,3,NaN,POINT (9.92884 49.75441)
...,...,...,...,...,...
8917704493,49.804200,9.918704,3,NaN,POINT (9.91870 49.80420)
8917704495,49.804163,9.918820,3,NaN,POINT (9.91882 49.80416)
8917704496,49.804228,9.918785,4,NaN,POINT (9.91878 49.80423)


In [4]:
nodes_df = gdf_nodes[["y", "x"]].copy()
nodes_df.columns = ["lat", "lng"]
nodes_df

,lat,lng
osmid,,
10799058,49.751452,9.926904
10799066,49.753819,9.928881
10799083,49.751798,9.927970
10799085,49.751011,9.927733
10799087,49.754406,9.928837
...,...,...
8917704493,49.804200,9.918704
8917704495,49.804163,9.918820
8917704496,49.804228,9.918785


In [5]:
sample_nodes_df = nodes_df.sample(n = 190, random_state= 3)

In [6]:
sample_nodes_df = sample_nodes_df.sort_values(by=["osmid"])
sample_nodes_df

,lat,lng
osmid,,
10799058,49.751452,9.926904
10799942,49.771879,9.940641
21273715,49.779945,9.937219
21287874,49.788421,9.952661
21287991,49.791405,9.996894
...,...,...
6034416208,49.791096,9.963971
6533691138,49.757266,9.945708
6895353620,49.813958,9.989616


In [7]:
J = sample_nodes_df.index.values
#J

In [8]:
sample_nodes_list = sample_nodes_df.values.tolist()
#sample_nodes_list

In [9]:
lat_DC = [49.7582904, 49.8610025, 49.8743879, 49.7726213, 49.7109099, 49.7682755, 49.7326234, 49.8113122, 49.7979646, 49.8109994]
lon_DC = [9.8039055, 9.8037194, 10.032524, 10.0631658, 10.0201193, 9.9394726, 9.9649484, 9.920716, 9.9426957, 9.9777457]
osmid = [283531080, 1312952369, 258899345, 28338982, 41891309, 36720636, 41891228, 148745748, 33171211, 21288066]

dc_tuples = list(zip(osmid, lat_DC, lon_DC))
set_of_all_DC = pd.DataFrame(dc_tuples, columns = ["osmid", "lat", "lng"])
set_of_all_DC.set_index("osmid", inplace=True)
I = set_of_all_DC.index.values
set_of_all_DC

,lat,lng
osmid,,
283531080,49.758290,9.803906
1312952369,49.861002,9.803719
258899345,49.874388,10.032524
28338982,49.772621,10.063166
41891309,49.710910,10.020119
36720636,49.768276,9.939473
41891228,49.732623,9.964948
148745748,49.811312,9.920716
33171211,49.797965,9.942696


In [10]:
joined_set = pd.concat([set_of_all_DC, sample_nodes_df], sort = False)
joined_set.head(15)

,lat,lng
osmid,,
283531080,49.758290,9.803906
1312952369,49.861002,9.803719
258899345,49.874388,10.032524
28338982,49.772621,10.063166
41891309,49.710910,10.020119
36720636,49.768276,9.939473
41891228,49.732623,9.964948
148745748,49.811312,9.920716
33171211,49.797965,9.942696


In [11]:
len(joined_set)

200

In [12]:
testlol = joined_set.to_dict("records")
testlol

[{'lat': 49.7582904, 'lng': 9.8039055},
 {'lat': 49.8610025, 'lng': 9.8037194},
 {'lat': 49.8743879, 'lng': 10.032524},
 {'lat': 49.7726213, 'lng': 10.0631658},
 {'lat': 49.7109099, 'lng': 10.0201193},
 {'lat': 49.7682755, 'lng': 9.9394726},
 {'lat': 49.7326234, 'lng': 9.9649484},
 {'lat': 49.8113122, 'lng': 9.920716},
 {'lat': 49.7979646, 'lng': 9.9426957},
 {'lat': 49.8109994, 'lng': 9.9777457},
 {'lat': 49.7514519, 'lng': 9.926904},
 {'lat': 49.7718794, 'lng': 9.9406408},
 {'lat': 49.7799446, 'lng': 9.9372194},
 {'lat': 49.7884206, 'lng': 9.9526609},
 {'lat': 49.7914048, 'lng': 9.9968944},
 {'lat': 49.7903125, 'lng': 9.9307793},
 {'lat': 49.7865819, 'lng': 9.9317883},
 {'lat': 49.7789385, 'lng': 9.9497211},
 {'lat': 49.7847581, 'lng': 9.9679458},
 {'lat': 49.783327, 'lng': 9.9558405},
 {'lat': 49.7855924, 'lng': 9.930666},
 {'lat': 49.7800078, 'lng': 9.9324318},
 {'lat': 49.7794253, 'lng': 9.9349845},
 {'lat': 49.7778329, 'lng': 9.933309},
 {'lat': 49.7827376, 'lng': 9.9315833},
 {'

In [13]:
len(testlol)

200

In [14]:
testlol15 = testlol[0:15]
testlol15

[{'lat': 49.7582904, 'lng': 9.8039055},
 {'lat': 49.8610025, 'lng': 9.8037194},
 {'lat': 49.8743879, 'lng': 10.032524},
 {'lat': 49.7726213, 'lng': 10.0631658},
 {'lat': 49.7109099, 'lng': 10.0201193},
 {'lat': 49.7682755, 'lng': 9.9394726},
 {'lat': 49.7326234, 'lng': 9.9649484},
 {'lat': 49.8113122, 'lng': 9.920716},
 {'lat': 49.7979646, 'lng': 9.9426957},
 {'lat': 49.8109994, 'lng': 9.9777457},
 {'lat': 49.7514519, 'lng': 9.926904},
 {'lat': 49.7718794, 'lng': 9.9406408},
 {'lat': 49.7799446, 'lng': 9.9372194},
 {'lat': 49.7884206, 'lng': 9.9526609},
 {'lat': 49.7914048, 'lng': 9.9968944}]

In [15]:
len(testlol15)

15

In [16]:
testlol100 = testlol[0:100]
len(testlol100)

100

In [17]:
testlol200 = testlol[100:200]
len(testlol200)

100

In [21]:
import requests, time

api_key = "NNQxEApTqU7HcNS8oTGT3cjdS87FNO6detvjjKL8Idc"

# "Profile mode (car fastest)" example request

#49.800460, 9.951607], [49.811422 ,9.981127

request = {
    "origins": testlol15,
    "destinations": testlol200,
    "regionDefinition": {"type": "autoCircle",
                        "margin": 40000},
    #"profile": "carFast",
    #"departureTime": ""
}

# Using a Session object allows you to persist certain parameters accross requests
# see: https://requests.readthedocs.io/en/master/user/advanced/
session = requests.Session()
# Add `?apiKey=xxxxxx` to all requests
session.params = {"apiKey": api_key}
# Raise an exception for any non 2xx or 3xx HTTP return codes
session.hooks = {"response": lambda r, *args, **kwargs: r.raise_for_status()}

# Send an asynchronous request, see: https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/topics/get-started/asynchronous-request.html
status_response = session.post(
    "https://matrix.router.hereapi.com/v8/matrix", json=request
).json()

# Polling for the job status for 100 times
# You might want to use exponential back-off instead of time.sleep
for _ in range(0, 100):
    # do not follow the redirect here
    status_response = session.get(
        status_response["statusUrl"], allow_redirects=False
    ).json()

    if status_response["status"] == "completed":
        # download the result
        matrix_response = session.get(status_response["resultUrl"]).json()
        print(matrix_response)
        break
   # elif status_response["status"] == "inProgress":
    #    continue
    else:
        print(f"An error occured: {status_response}")
        break
    time.sleep(1)  # sleep for 500 ms

An error occured: {'matrixId': '32466d05-4c5d-4177-8f73-30ab8da12ce1', 'status': 'inProgress'}


In [19]:
#dict_test = response.as_dict()
mylist = matrix_response["matrix"]["travelTimes"]
np_array = np.array(mylist)
reshaped_array = np.reshape(np_array, (15,100))
mydf = pd.DataFrame(reshaped_array)
mydf

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,1039,1778,1159,1100,913,955,1192,1091,1268,...,1031,1090,1067,1347,1048,984,1209,1100,879,1334
1,1042,0,1770,1594,1781,1434,1636,1034,1128,1260,...,1552,1611,1588,1295,1569,1551,1504,949,721,1287
2,1660,1819,0,938,1412,1072,1322,1285,833,608,...,1190,1249,1226,833,1207,1189,1300,811,1080,827
3,1311,1584,1041,0,762,895,1018,1050,598,531,...,1013,1072,1049,501,1030,1012,1123,576,845,260
4,1202,1862,1561,684,0,731,706,1524,1039,1051,...,823,882,859,1130,840,839,1069,1052,1319,944
5,773,1421,993,713,570,0,435,883,471,483,...,261,287,297,562,278,204,338,484,678,549
6,954,1614,1390,833,754,525,0,1327,868,880,...,507,556,572,959,473,596,782,881,1122,946
7,1191,1081,1351,1175,1435,1015,1315,0,709,841,...,1133,1192,1169,876,1150,1132,1085,530,320,868
8,1149,1120,842,666,876,536,786,586,0,332,...,654,713,690,363,671,653,749,112,381,359
9,1199,1358,593,679,951,611,861,824,372,0,...,729,788,765,222,746,728,839,350,619,372


In [19]:
len(J)

200

In [21]:
nn = np.concatenate((I,J), axis = None)
new_colnames = nn.tolist()

In [22]:
cols = new_colnames[0:100]
ix = new_colnames[0:15]

In [25]:
my_df1 = mydf
my_df1.columns = cols
my_df1.index = ix
my_df1.rename_axis("osmid")
my_df1

,283531080,1312952369,258899345,28338982,41891309,36720636,41891228,148745748,33171211,21288066,...,251053298,251053408,251059033,253089137,254745660,254747320,254750302,257042823,259407871,259448553
283531080,0,1039,1778,1159,1100,913,955,1192,1091,1268,...,1031,1090,1067,1347,1048,984,1209,1100,879,1334
1312952369,1042,0,1770,1594,1781,1434,1636,1034,1128,1260,...,1552,1611,1588,1295,1569,1551,1504,949,721,1287
258899345,1660,1819,0,938,1412,1072,1322,1285,833,608,...,1190,1249,1226,833,1207,1189,1300,811,1080,827
28338982,1311,1584,1041,0,762,895,1018,1050,598,531,...,1013,1072,1049,501,1030,1012,1123,576,845,260
41891309,1202,1862,1561,684,0,731,706,1524,1039,1051,...,823,882,859,1130,840,839,1069,1052,1319,944
36720636,773,1421,993,713,570,0,435,883,471,483,...,261,287,297,562,278,204,338,484,678,549
41891228,954,1614,1390,833,754,525,0,1327,868,880,...,507,556,572,959,473,596,782,881,1122,946
148745748,1191,1081,1351,1175,1435,1015,1315,0,709,841,...,1133,1192,1169,876,1150,1132,1085,530,320,868
33171211,1149,1120,842,666,876,536,786,586,0,332,...,654,713,690,363,671,653,749,112,381,359
21288066,1199,1358,593,679,951,611,861,824,372,0,...,729,788,765,222,746,728,839,350,619,372


In [26]:
my_df1.to_hdf("15_100.h5", mode= "w", key = "my_df1")

In [27]:
my_df1.to_csv("15_100.csv")

In [5]:
avoid = Avoid(
    features=[AvoidFeature.toll_road],
    areas=[AvoidArea(north=30, south=45, west=30, east=45)],
)
truck = Truck(
    shipped_hazardous_goods=[ShippedHazardousGood.gas],
    gross_weight=750,
    weight_per_axle=100,
    height=2000,
    width=350,
    length=10000,
    tunnel_category=TunnelCategory.c,
    axle_count=5,
    truck_type=TruckType.tractor,
    trailer_count=5,
)
response = routing_api.sync_matrix(
    origins=[[9.933231, -84.076831]],
    destinations=[[9.934574, -84.065544]],
    matrix_type=MatrixRoutingType.circle,
    center=[9.933300, -84.066891],
    radius=10000,
    avoid=avoid,
    truck=truck,
)
print(response.as_dict())

{'matrixId': '93b40cf4-94a0-4d3e-9fa2-686415940fa5', 'matrix': {'numOrigins': 1, 'numDestinations': 1, 'travelTimes': [290]}, 'regionDefinition': {'type': 'circle', 'center': {'lat': 9.9333, 'lng': -84.066891}, 'radius': 10000}}


In [12]:
response.as_dict()

{'matrixId': '93b40cf4-94a0-4d3e-9fa2-686415940fa5',
 'matrix': {'numOrigins': 1, 'numDestinations': 1, 'travelTimes': [290]},
 'regionDefinition': {'type': 'circle',
  'center': {'lat': 9.9333, 'lng': -84.066891},
  'radius': 10000}}

In [14]:
response = routing_api.sync_matrix(
    origins=[[49.800460, 9.951607]],
    destinations=[[49.811422 ,9.981127]],
    matrix_type=MatrixRoutingType.circle,
    center=[49.793421, 9.946744],
    radius=10000
)
print(response.as_dict())

{'matrixId': '00f1348d-0538-47e8-9984-e19472ddb3fb', 'matrix': {'numOrigins': 1, 'numDestinations': 1, 'travelTimes': [400]}, 'regionDefinition': {'type': 'circle', 'center': {'lat': 49.793421, 'lng': 9.946744}, 'radius': 10000}}


In [15]:
print(response.as_list())

AttributeError: 'RoutingMatrixResponse' object has no attribute 'as_list'

In [16]:
response.as_dict()

{'matrixId': '00f1348d-0538-47e8-9984-e19472ddb3fb',
 'matrix': {'numOrigins': 1, 'numDestinations': 1, 'travelTimes': [400]},
 'regionDefinition': {'type': 'circle',
  'center': {'lat': 49.793421, 'lng': 9.946744},
  'radius': 10000}}

In [18]:
origin_list = [49.800460, 9.951607], [49.811422 ,9.981127]

In [19]:
origin_list

([49.80046, 9.951607], [49.811422, 9.981127])

In [20]:
dest_list = origin_list
dest_list

([49.80046, 9.951607], [49.811422, 9.981127])

In [21]:
response = routing_api.sync_matrix(
    origins= origin_list,
    destinations = dest_list,
    matrix_type=MatrixRoutingType.circle,
    center=[49.793421, 9.946744],
    radius=10000
)
response.as_dict()

{'matrixId': 'af8bed38-831a-4c5f-aa90-949cfe6a23b7',
 'matrix': {'numOrigins': 2,
  'numDestinations': 2,
  'travelTimes': [0, 401, 297, 0]},
 'regionDefinition': {'type': 'circle',
  'center': {'lat': 49.793421, 'lng': 9.946744},
  'radius': 10000}}

In [25]:
response("travelTimes")

TypeError: 'RoutingMatrixResponse' object is not callable

In [26]:
dict_test = response.as_dict()

In [31]:
dict_test.keys()


dict_keys(['matrixId', 'matrix', 'regionDefinition'])

In [32]:
dict_test

{'matrixId': 'af8bed38-831a-4c5f-aa90-949cfe6a23b7',
 'matrix': {'numOrigins': 2,
  'numDestinations': 2,
  'travelTimes': [0, 401, 297, 0]},
 'regionDefinition': {'type': 'circle',
  'center': {'lat': 49.793421, 'lng': 9.946744},
  'radius': 10000}}

In [36]:
mylist = dict_test["matrix"]["travelTimes"]
type(mylist)

list

In [37]:
mylist

[0, 401, 297, 0]

In [39]:
import numpy as np
import pandas as pd

In [41]:
np_array = np.array(mylist)
reshaped_array = np.reshape(np_array, (2,2))
mydf = pd.DataFrame(reshaped_array)
mydf

,0,1
0,0,401
1,297,0


In [1]:
import requests, time

In [5]:
import requests, time

api_key = "NNQxEApTqU7HcNS8oTGT3cjdS87FNO6detvjjKL8Idc"

# "Profile mode (car fastest)" example request

#49.800460, 9.951607], [49.811422 ,9.981127

request = {
    "origins": [{"lat": 49.800460, "lng": 9.951607}, {"lat": 49.811422, "lng": 9.981127}],
#    "destinations": [
#        {"lat": 51.3397, "lng": 12.3731},
#        {"lat": 51.0504, "lng": 13.7373},
#    ],
    "regionDefinition": {"type": "autoCircle",
                        "margin": 10000},
    #"profile": "carFast",
    #"departureTime": ""
}

# Using a Session object allows you to persist certain parameters accross requests
# see: https://requests.readthedocs.io/en/master/user/advanced/
session = requests.Session()
# Add `?apiKey=xxxxxx` to all requests
session.params = {"apiKey": api_key}
# Raise an exception for any non 2xx or 3xx HTTP return codes
session.hooks = {"response": lambda r, *args, **kwargs: r.raise_for_status()}

# Send an asynchronous request, see: https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/topics/get-started/asynchronous-request.html
status_response = session.post(
    "https://matrix.router.hereapi.com/v8/matrix", json=request
).json()

# Polling for the job status for 100 times
# You might want to use exponential back-off instead of time.sleep
for _ in range(0, 100):
    # do not follow the redirect here
    status_response = session.get(
        status_response["statusUrl"], allow_redirects=False
    ).json()

    if status_response["status"] == "completed":
        # download the result
        matrix_response = session.get(status_response["resultUrl"]).json()
        print(matrix_response)
        break
    elif status_response["accepted"] or status_response["inProgress"]:
        continue
    else:
        print(f"An error occured: {status_response}")
        break
    time.sleep(0.5)  # sleep for 500 ms


{'matrixId': 'b8666171-1290-488d-b436-b145b2146136', 'matrix': {'numOrigins': 2, 'numDestinations': 2, 'travelTimes': [0, 530, 306, 0]}, 'regionDefinition': {'type': 'circle', 'center': {'lat': 49.805941, 'lng': 9.966367}, 'radius': 11223}}


In [ ]:
'travelTimes': [0, 533, 307, 0]}

In [23]:
import requests, time

api_key = "NNQxEApTqU7HcNS8oTGT3cjdS87FNO6detvjjKL8Idc"

# "Profile mode (car fastest)" example request

#49.800460, 9.951607], [49.811422 ,9.981127

request = {
    "origins": testlol,
#    "destinations": [
#        {"lat": 51.3397, "lng": 12.3731},
#        {"lat": 51.0504, "lng": 13.7373},
#    ],
    "regionDefinition": {"type": "autoCircle",
                        "margin": 45000},
    #"profile": "carFast",
    #"departureTime": ""
}

# Using a Session object allows you to persist certain parameters accross requests
# see: https://requests.readthedocs.io/en/master/user/advanced/
session = requests.Session()
# Add `?apiKey=xxxxxx` to all requests
session.params = {"apiKey": api_key}
# Raise an exception for any non 2xx or 3xx HTTP return codes
session.hooks = {"response": lambda r, *args, **kwargs: r.raise_for_status()}

# Send an asynchronous request, see: https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/topics/get-started/asynchronous-request.html
status_response = session.post(
    "https://matrix.router.hereapi.com/v8/matrix", json=request
).json()

# Polling for the job status for 100 times
# You might want to use exponential back-off instead of time.sleep
for _ in range(0, 100):
    # do not follow the redirect here
    status_response = session.get(
        status_response["statusUrl"], allow_redirects=False
    ).json()

    if status_response["status"] == "completed":
        # download the result
        matrix_response = session.get(status_response["resultUrl"]).json()
        print(matrix_response)
        break
    elif status_response["accepted"] or status_response["inProgress"]:
        continue
    else:
        print(f"An error occured: {status_response}")
        break
    time.sleep(0.5)  # sleep for 500 ms


{'matrixId': '076449f6-3ab8-4cab-b305-334bcc752dd5', 'matrix': {'numOrigins': 10, 'numDestinations': 10, 'travelTimes': [0, 1075, 1969, 1837, 1399, 954, 1019, 1154, 1267, 1471, 1026, 0, 2081, 1852, 1876, 1503, 1660, 1024, 1156, 1510, 1746, 1804, 0, 945, 1375, 1128, 1415, 1282, 836, 625, 1309, 1532, 998, 0, 761, 902, 1020, 1010, 564, 500, 1214, 1942, 1668, 775, 0, 708, 688, 1612, 1106, 1275, 803, 1432, 1244, 1012, 574, 0, 472, 904, 570, 746, 960, 1688, 1627, 1395, 732, 512, 0, 1332, 953, 1129, 1192, 1076, 1599, 1443, 1467, 1094, 1381, 0, 747, 1101, 1206, 1116, 961, 789, 895, 551, 838, 594, 0, 463, 1265, 1323, 584, 673, 991, 647, 934, 801, 355, 0]}, 'regionDefinition': {'type': 'circle', 'center': {'lat': 49.7926489, 'lng': 9.9334426}, 'radius': 22015}}


In [26]:
mylist = matrix_response["matrix"]["travelTimes"]
mylist

[0,
 1075,
 1969,
 1837,
 1399,
 954,
 1019,
 1154,
 1267,
 1471,
 1026,
 0,
 2081,
 1852,
 1876,
 1503,
 1660,
 1024,
 1156,
 1510,
 1746,
 1804,
 0,
 945,
 1375,
 1128,
 1415,
 1282,
 836,
 625,
 1309,
 1532,
 998,
 0,
 761,
 902,
 1020,
 1010,
 564,
 500,
 1214,
 1942,
 1668,
 775,
 0,
 708,
 688,
 1612,
 1106,
 1275,
 803,
 1432,
 1244,
 1012,
 574,
 0,
 472,
 904,
 570,
 746,
 960,
 1688,
 1627,
 1395,
 732,
 512,
 0,
 1332,
 953,
 1129,
 1192,
 1076,
 1599,
 1443,
 1467,
 1094,
 1381,
 0,
 747,
 1101,
 1206,
 1116,
 961,
 789,
 895,
 551,
 838,
 594,
 0,
 463,
 1265,
 1323,
 584,
 673,
 991,
 647,
 934,
 801,
 355,
 0]

In [41]:
matrix_response

{'matrixId': '4d06d5ae-dbc2-493c-b8e3-7fcf105cc5a9',
 'matrix': {'numOrigins': 2,
  'numDestinations': 2,
  'travelTimes': [7219, 7518, 13062, 14978]},
 'regionDefinition': {'type': 'world'}}

In [46]:
import requests, time

api_key = "NNQxEApTqU7HcNS8oTGT3cjdS87FNO6detvjjKL8Idc"

# "Profile mode (car fastest)" example request
request = {
    "origins": [{"lat": 52.5309, "lng": 13.3849}, {"lat": 54.0924, "lng": 12.0991}],
    "destinations": [
        {"lat": 51.3397, "lng": 12.3731},
        {"lat": 51.0504, "lng": 13.7373},
    ],
    "regionDefinition": {"type": "autoCircle",
                        "margin": 40000},
    #"profile": "carFast",
    #"departureTime": "2020-12-19T18:23:45+0100"
}

# Using a Session object allows you to persist certain parameters accross requests
# see: https://requests.readthedocs.io/en/master/user/advanced/
session = requests.Session()
# Add `?apiKey=xxxxxx` to all requests
session.params = {"apiKey": api_key}
# Raise an exception for any non 2xx or 3xx HTTP return codes
session.hooks = {"response": lambda r, *args, **kwargs: r.raise_for_status()}

# Send an asynchronous request, see: https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/topics/get-started/asynchronous-request.html
status_response = session.post(
    "https://matrix.router.hereapi.com/v8/matrix", json=request
).json()

# Polling for the job status for 100 times
# You might want to use exponential back-off instead of time.sleep
for _ in range(0, 100):
    # do not follow the redirect here
    status_response = session.get(
        status_response["statusUrl"], allow_redirects=False
    ).json()

    if status_response["status"] == "completed":
        # download the result
        matrix_response = session.get(status_response["resultUrl"]).json()
        print(matrix_response)
        break
    elif status_response["accepted"] or status_response["inProgress"]:
        continue
    else:
        print(f"An error occured: {status_response}")
        break
    time.sleep(0.5)  # sleep for 500 ms


HTTPError: 400 Client Error: Bad Request for url: https://matrix.router.hereapi.com/v8/matrix?apiKey=NNQxEApTqU7HcNS8oTGT3cjdS87FNO6detvjjKL8Idc

In [47]:
import requests, time

api_key = "NNQxEApTqU7HcNS8oTGT3cjdS87FNO6detvjjKL8Idc"

# "Profile mode (car fastest)" example request
request = {
    "origins": [{"lat": 52.5309, "lng": 13.3849}, {"lat": 54.0924, "lng": 12.0991}],
    "destinations": [
        {"lat": 51.3397, "lng": 12.3731},
        {"lat": 51.0504, "lng": 13.7373},
    ],
    "regionDefinition": {"type": "autoCircle"}
    #"profile": "carFast",
}

# Using a Session object allows you to persist certain parameters accross requests
# see: https://requests.readthedocs.io/en/master/user/advanced/
session = requests.Session()
# Add `?apiKey=xxxxxx` to all requests
session.params = {"apiKey": api_key}
# Raise an exception for any non 2xx or 3xx HTTP return codes
session.hooks = {"response": lambda r, *args, **kwargs: r.raise_for_status()}

# Send an asynchronous request, see: https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/topics/get-started/asynchronous-request.html
status_response = session.post(
    "https://matrix.router.hereapi.com/v8/matrix?async=false", json=request
).json()

# Polling for the job status for 100 times
# You might want to use exponential back-off instead of time.sleep
for _ in range(0, 100):
    # do not follow the redirect here
    status_response = session.get(
        status_response["statusUrl"], allow_redirects=False
    ).json()

    if status_response["status"] == "completed":
        # download the result
        matrix_response = session.get(status_response["resultUrl"]).json()
        print(matrix_response)
        break
    elif status_response["accepted"] or status_response["inProgress"]:
        continue
    else:
        print(f"An error occured: {status_response}")
        break
    time.sleep(0.5)  # sleep for 500 ms


KeyError: 'statusUrl'

In [28]:
import requests, time

api_key = "NNQxEApTqU7HcNS8oTGT3cjdS87FNO6detvjjKL8Idc"

# "Profile mode (car fastest)" example request

#49.800460, 9.951607], [49.811422 ,9.981127

request = {
    "origins": testlol15,
    "destinations": testlol200,
    "regionDefinition": {"type": "autoCircle",
                        "margin": 40000},
    #"profile": "carFast",
    #"departureTime": ""
}

# Using a Session object allows you to persist certain parameters accross requests
# see: https://requests.readthedocs.io/en/master/user/advanced/
session = requests.Session()
# Add `?apiKey=xxxxxx` to all requests
session.params = {"apiKey": api_key}
# Raise an exception for any non 2xx or 3xx HTTP return codes
session.hooks = {"response": lambda r, *args, **kwargs: r.raise_for_status()}

# Send an asynchronous request, see: https://developer.here.com/documentation/matrix-routing-api/8.3.0/dev_guide/topics/get-started/asynchronous-request.html
status_response = session.post(
    "https://matrix.router.hereapi.com/v8/matrix?async=false", json=request
).json()

# Polling for the job status for 100 times
# You might want to use exponential back-off instead of time.sleep
for _ in range(0, 100):
    # do not follow the redirect here
    status_response = session.get(
        status_response["statusUrl"], allow_redirects=True
    ).json()

    if status_response["status"] == "completed":
        # download the result
        matrix_response = session.get(status_response["resultUrl"]).json()
        print(matrix_response)
        break
    elif status_response["inProgress"]:
        continue
    else:
        print(f"An error occured: {status_response}")
        break
    time.sleep(1)  # sleep for 500 ms

KeyError: 'statusUrl'